In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, cohen_kappa_score
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import warnings
warnings.filterwarnings('ignore')
matplotlib.rcParams["figure.facecolor"] = 'FFFFFF'
matplotlib.rcParams['figure.dpi'] = 100

In [34]:
data = pd.read_excel('online_retail_II.xlsx', [0, 1])

In [35]:
data = pd.concat([data[0], data[1]], ignore_index=True)

In [36]:
data.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [37]:
print(len(data))
data['Quantity'] = data['Quantity'].abs()
# data = data.drop_duplicates(subset=['StockCode', 'Quantity', 'Price', 'Customer ID'])
print(len(data))

1067371
1067371


In [ ]:
print(data.columns)
pairs = 0
for i, row in enumerate(data[data['Invoice'].str[0] == 'C'].values):
    duplicates_found = data[data['StockCode'] == row[1]][data['Customer ID'] == row[6]][data['Quantity'] == -row[3]]
    if(len(duplicates_found) != 2):
        print(f'#{i}')
        print(row)
        print(duplicates_found)
        print(f"{pairs} / {i}")
        print('\n\n\n')
    else:
        pairs += 1

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Cancelled'],
      dtype='object')
#0
['C489449' 22087 'PAPER BUNTING WHITE LACE' -12
 Timestamp('2009-12-01 10:33:00') 2.95 16321.0 'Australia' True]
    Invoice StockCode               Description  Quantity         InvoiceDate  \
187  489450     22087  PAPER BUNTING WHITE LACE        12 2009-12-01 10:36:00   

     Price  Customer ID    Country  Cancelled  
187   2.95      16321.0  Australia      False  
0 / 0




#1
['C489449' '85206A' 'CREAM FELT EASTER EGG BASKET' -6
 Timestamp('2009-12-01 10:33:00') 1.65 16321.0 'Australia' True]
    Invoice StockCode                   Description  Quantity  \
188  489450    85206A  CREAM FELT EASTER EGG BASKET         6   

            InvoiceDate  Price  Customer ID    Country  Cancelled  
188 2009-12-01 10:36:00   1.65      16321.0  Australia      False  
0 / 1




#2
['C489449' 21895 "POTTING SHED SOW 'N' GROW SET" -4
 Timestamp

KeyboardInterrupt: 

In [38]:
data['Cancelled'] = (data['Invoice'].str[0] == 'C')

In [39]:
data.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Cancelled'],
      dtype='object')

In [40]:
data['Cancelled'].value_counts()
data[['Invoice', 'Cancelled']].drop_duplicates()['Cancelled'].value_counts()

False    45336
True      8292
Name: Cancelled, dtype: int64

In [41]:
data['StockCode'].value_counts()

85123A       5829
22423        4424
85099B       4216
21212        3318
20725        3259
             ... 
DCGS0067        1
DCGS0066P       1
DCGS0071        1
DCGS0073        1
90197A          1
Name: StockCode, Length: 5305, dtype: int64

In [86]:
grouped_data = data.groupby('Invoice')
print(grouped_data.head())

        Invoice StockCode                          Description  Quantity  \
0        489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1        489434    79323P                   PINK CHERRY LIGHTS        12   
2        489434    79323W                  WHITE CHERRY LIGHTS        12   
3        489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4        489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
...         ...       ...                                  ...       ...   
1067355  581587     22631             CIRCUS PARADE LUNCH BOX         12   
1067356  581587     22556       PLASTERS IN TIN CIRCUS PARADE         12   
1067357  581587     22555            PLASTERS IN TIN STRONGMAN        12   
1067358  581587     22728            ALARM CLOCK BAKELIKE PINK         4   
1067359  581587     22727            ALARM CLOCK BAKELIKE RED          4   

                InvoiceDate  Price  Customer ID         Country  Cancelled  
0       20

In [193]:
aggregated = {
    "Invoice": [],
    "EarliestInvoiceDate": [],
    "LatestInvoiceDate": [],
    "PriceSum": [],
    "TotalQuantity": [],
    "DifferentProducts": [],
    
    "Manual": [],
    "Discount": [],
    "ManualSum": [],
    "DiscountSum": [],
    "Cancelled": [],
    
    "isBritish": [],

    "MostCommon1Quantity": [],
    "MostCommon2Quantity": [],
    "MostCommon3Quantity": [],
    "MostCommon1Price": [],
    "MostCommon2Price": [],
    "MostCommon3Price": [],

    "MostExpensive1Price": [],
    "MostExpensive2Price": [],
    "MostExpensive3Price": [],
    "MostExpensive1Quantity": [],
    "MostExpensive2Quantity": [],
    "MostExpensive3Quantity": []


}

for group_name, df_group in grouped_data:
    if all(df_group['StockCode'].isin(['M', 'm', 'S', 'B', 'D'])):
        continue
    assert not any(df_group['StockCode'] == 'S')

    products = ~df_group['StockCode'].isin(['M', 'm', 'D'])

    assert all(df_group['Invoice'] == group_name)
    aggregated['Invoice'].append(group_name)

    aggregated['isBritish'].append(df_group['Country'].iloc[0] == 'United Kingdom')
    
    group_invoice_dates = df_group['InvoiceDate'].sort_values()
    aggregated['EarliestInvoiceDate'].append(group_invoice_dates.iloc[0])
    aggregated['LatestInvoiceDate'].append(group_invoice_dates.iloc[-1])

    aggregated['PriceSum'].append((df_group['Price'] * df_group['Quantity']).sum())
    aggregated['TotalQuantity'].append(df_group['Quantity'].sum())
    aggregated['DifferentProducts'].append(products.value_counts()[True])

    sorted_group = df_group[products].sort_values(by='Quantity',  ascending=True)
    aggregated['MostCommon1Quantity'].append(sorted_group['Quantity'].iloc[0])
    aggregated['MostCommon1Price'].append(sorted_group['Price'].iloc[0])
    if len(sorted_group) > 2:
        aggregated['MostCommon3Quantity'].append(sorted_group['Quantity'].iloc[2])
        aggregated['MostCommon3Price'].append(sorted_group['Price'].iloc[2])

    else:
        aggregated['MostCommon3Quantity'].append(0)
        aggregated['MostCommon3Price'].append(0)

    if len(sorted_group) > 1:
        aggregated['MostCommon2Quantity'].append(sorted_group['Quantity'].iloc[1])
        aggregated['MostCommon2Price'].append(sorted_group['Price'].iloc[1])

    else:
        aggregated['MostCommon2Quantity'].append(0)
        aggregated['MostCommon2Price'].append(0)

    sorted_by_prices = df_group[products].sort_values(by='Price', ascending=True)
    aggregated['MostExpensive1Price'].append(sorted_by_prices['Price'].iloc[0])
    aggregated['MostExpensive1Quantity'].append(sorted_by_prices['Quantity'].iloc[0])
    if len(sorted_by_prices) > 2:
        aggregated['MostExpensive3Price'].append(sorted_by_prices['Price'].iloc[2])
        aggregated['MostExpensive3Quantity'].append(sorted_by_prices['Quantity'].iloc[2])
    else:
        aggregated['MostExpensive3Price'].append(0)
        aggregated['MostExpensive3Quantity'].append(0)
    if len(sorted_by_prices) > 1:
        aggregated['MostExpensive2Price'].append(sorted_by_prices['Price'].iloc[1])
        aggregated['MostExpensive2Quantity'].append(sorted_by_prices['Quantity'].iloc[1])
    else:
        aggregated['MostExpensive2Price'].append(0)
        aggregated['MostExpensive2Quantity'].append(0)

    aggregated['Manual'].append(0)
    aggregated['Discount'].append(0)
    aggregated['ManualSum'].append(0)
    aggregated['DiscountSum'].append(0)

    stock_code_idx = list(df_group.columns).index('StockCode')
    price_idx = list(df_group.columns).index('Price')
    quantity_idx = list(df_group.columns).index('Price')
    
    for row in df_group.values:
        if row[stock_code_idx] in {'M', 'm'}:
            aggregated['Manual'][-1] += 1
            aggregated['ManualSum'][-1] += row[price_idx] * row[quantity_idx]
        elif row[stock_code_idx] == 'D':
            aggregated['Discount'][-1] += 1
            aggregated['DiscountSum'][-1] += row[price_idx] * row[quantity_idx]
        
    aggregated['PriceSum'][-1] -= aggregated['DiscountSum'][-1] * 2

    aggregated['Cancelled'].append(str(group_name)[0] == 'C')

In [194]:
old_agg_data = pd.DataFrame(aggregated)
old_agg_data.columns

Index(['Invoice', 'EarliestInvoiceDate', 'LatestInvoiceDate', 'PriceSum',
       'TotalQuantity', 'DifferentProducts', 'Manual', 'Discount', 'ManualSum',
       'DiscountSum', 'Cancelled', 'isBritish', 'MostCommon1Quantity',
       'MostCommon2Quantity', 'MostCommon3Quantity', 'MostCommon1Price',
       'MostCommon2Price', 'MostCommon3Price', 'MostExpensive1Price',
       'MostExpensive2Price', 'MostExpensive3Price', 'MostExpensive1Quantity',
       'MostExpensive2Quantity', 'MostExpensive3Quantity'],
      dtype='object')

In [195]:
print(len(old_agg_data))
print(len(old_agg_data.drop_duplicates(subset=['PriceSum', 'TotalQuantity'], keep=False)))
# old_agg_data = old_agg_data.drop_duplicates(subset=['PriceSum'], keep=False)
# old_agg_data = old_agg_data.drop_duplicates(subset=['TotalQuantity'], keep=False)
print(len(old_agg_data))
agg_data = old_agg_data

# agg_data = old_agg_data.drop_duplicates(subset=['PriceSum', 'TotalQuantity'], keep=False)

52734
38203
52734


In [196]:
ready_data = agg_data.drop(columns=['Invoice', 'EarliestInvoiceDate', 'LatestInvoiceDate'])
# 'Manual', 'ManualSum', 'Discount', 'DiscountSum', 'EarliestInvoiceDate', 'LatestInvoiceDate', 'PriceSum'
ready_data['InvoiceDate'] = agg_data['EarliestInvoiceDate'].dt.month * 31 + agg_data['EarliestInvoiceDate'].dt.day
ready_data['InvoiceHour'] = agg_data['EarliestInvoiceDate'].dt.hour

In [197]:
train_data, valid_data = train_test_split(ready_data, random_state=421)
train_X, train_y = train_data.drop(columns=['Cancelled']), train_data['Cancelled']
valid_X, valid_y = valid_data.drop(columns=['Cancelled']), valid_data['Cancelled']

In [198]:
# plt.hist(train_data['TimeSpent'], bins=list(range(0, 20, 1)));

In [199]:
train_data.corr()

,PriceSum,TotalQuantity,DifferentProducts,Manual,Discount,ManualSum,DiscountSum,Cancelled,isBritish,MostCommon1Quantity,...,MostCommon2Price,MostCommon3Price,MostExpensive1Price,MostExpensive2Price,MostExpensive3Price,MostExpensive1Quantity,MostExpensive2Quantity,MostExpensive3Quantity,InvoiceDate,InvoiceHour
PriceSum,1.000000,0.680329,0.278564,0.005138,-0.058069,0.010433,-0.083942,-0.069726,-0.054459,0.676048,...,0.152932,0.054466,0.189135,0.148760,-0.006833,0.686762,0.182300,0.173105,0.030127,-0.022648
TotalQuantity,0.680329,1.000000,0.127947,0.001498,0.000260,-0.001688,0.003565,-0.072779,-0.067768,0.593928,...,-0.001143,0.017806,-0.006776,-0.000931,-0.033454,0.701327,0.550961,0.516425,0.010259,-0.024220
DifferentProducts,0.278564,0.127947,1.000000,0.059230,-0.003453,0.006486,-0.001878,-0.174802,0.004367,-0.021543,...,0.001281,0.106957,-0.014142,-0.007108,-0.064715,-0.020738,-0.006133,0.002171,0.048974,0.042190
Manual,0.005138,0.001498,0.059230,1.000000,-0.001021,0.112710,-0.000784,-0.000392,0.020615,-0.004123,...,-0.000530,-0.001402,-0.002482,-0.000582,-0.010331,-0.005258,-0.006872,-0.005768,0.002803,0.021623
Discount,-0.058069,0.000260,-0.003453,-0.001021,1.000000,-0.000150,0.662280,0.025596,-0.009651,0.000344,...,-0.000324,0.020004,-0.000308,-0.000143,0.002845,0.000201,0.002401,0.000070,0.000711,0.008754
ManualSum,0.010433,-0.001688,0.006486,0.112710,-0.000150,1.000000,-0.000115,0.000605,0.004073,-0.000679,...,-0.000608,-0.003847,-0.000057,-0.000303,-0.006117,-0.001128,-0.002014,-0.001784,0.002315,0.001888
DiscountSum,-0.083942,0.003565,-0.001878,-0.000784,0.662280,-0.000115,1.000000,0.019675,-0.014455,0.000069,...,-0.000265,0.054360,-0.000252,-0.000122,-0.000100,0.000289,0.008600,0.003021,0.005997,0.004564
Cancelled,-0.069726,-0.072779,-0.174802,-0.000392,0.025596,0.000605,0.019675,1.000000,-0.051371,-0.004987,...,0.019635,-0.079099,0.061045,0.029068,0.007823,-0.017153,-0.046689,-0.054042,-0.009605,0.060512
isBritish,-0.054459,-0.067768,0.004367,0.020615,-0.009651,0.004073,-0.014455,-0.051371,1.000000,0.008804,...,-0.002238,-0.034984,0.004731,0.002864,-0.012560,-0.000216,-0.027365,-0.032300,-0.006725,0.070215
MostCommon1Quantity,0.676048,0.593928,-0.021543,-0.004123,0.000344,-0.000679,0.000069,-0.004987,0.008804,1.000000,...,-0.002033,-0.012762,-0.001513,-0.000982,-0.018461,0.972137,0.097362,0.058006,0.006700,-0.009269


In [200]:
train_data['InvoiceDate'].value_counts()

373    348
374    339
380    334
345    283
379    276
      ... 
127     15
156     15
75      13
106     12
68       9
Name: InvoiceDate, Length: 352, dtype: int64

In [201]:
model = xgb.XGBClassifier()
model.fit(train_X, train_y)
pred = model.predict(valid_X)
# print(train_y)

print(cohen_kappa_score(valid_y, pred))
print(accuracy_score(valid_y, pred))
# print(export_text(model))

0.71752209008262
0.9304459951456311


In [202]:
print(len(valid_y))

13184


In [203]:
rand = (np.random.uniform(0, 1, len(valid_y)) > train_data['Cancelled'].value_counts()[False] / len(train_data['Cancelled'])).astype(np.int64)
print(cohen_kappa_score(valid_y, rand))
print(accuracy_score(valid_y, rand))

0.010537152945405692
0.7580400485436893


In [204]:
zeros = np.zeros(len(valid_y))
print(cohen_kappa_score(valid_y, zeros))
print(accuracy_score(valid_y, zeros))

0.0
0.8596783980582524
